# Importing the libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("newly_sampled_data.csv")

FileNotFoundError: [Errno 2] File newly_sampled_data.csv does not exist: 'newly_sampled_data.csv'

# Analysis

In [ ]:
len(df)

In [ ]:
df.shape

In [ ]:
df.head()

# INITIAL EDA

In [ ]:
a=df.type.value_counts()

In [ ]:
a

In [ ]:
a.plot(kind="bar",figsize=(14,8))

plt.xlabel("news category")
plt.ylabel("total no. of counts")
plt.title("counts for different news categories")
plt.savefig("newstypecount.png")

In [ ]:
b=df.domain.value_counts()

In [ ]:
b.nlargest(30)

In [ ]:
d=b.nlargest(10).plot(kind="bar",color="goldenrod",figsize=((14,8)))
plt.xlabel("domain_category")
plt.ylabel("total_counts")
plt.title("counts for domain", fontsize=24)

In [ ]:
df.head()

In [ ]:
nytimes=df.loc[df["domain"]=="nytimes.com"]
nytimes.type.value_counts().plot(kind='bar')
plt.xlabel("news_category")
plt.ylabel("counts")
plt.title("Type of news count for nytimes.com")


### The dataset defines all the documents from the New York times as reliable

In [ ]:
beforeitsnews=df.loc[df["domain"]=='beforeitsnews.com']

In [ ]:
beforeitsnews.type.value_counts().plot(kind="bar",color="goldenrod")
plt.xlabel("news_category")
plt.ylabel("total counts")
plt.title("type of news and its count for beforeitsnews.com")

### The dataset defines all the documents from the "beforeitsnews" as fake

# Binary labels

### "fake" vs "reliable" news in the dataset

In [ ]:
import nltk

In [ ]:
S=pd.read_csv("fake+real.csv")

In [ ]:
S.shape

In [ ]:
S.head()

In [ ]:
S.type.value_counts().plot(kind="bar",figsize=((8,5)))
plt.xlabel("type_category")
plt.ylabel("total_counts")
plt.title("type_category and its total count")

In [ ]:
S.isnull().values.any()

In [ ]:
S.isnull().sum().plot(kind="bar",ylim=(0,len(S)),title ="NAN values per columns")

In [ ]:
S.columns

In [ ]:
S=S.drop(columns=["summary","tags","keywords"])

In [ ]:
S.head()

In [ ]:
S.shape

In [ ]:
### Now removing the irrelevant features such as id, scraped_at,updated_at,inserted_at and unnamed: 0

In [ ]:
S=S.drop(columns=["Unnamed: 0","id","inserted_at","updated_at","scraped_at"])

In [ ]:
S.head()

In [ ]:
import seaborn as sns
S.domain.value_counts().plot(kind="pie",labels=None)
plt.text(-0.75, 0.35, 'nytimes.com', dict(size=15))
plt.text(-0.75, -0.6, 'beforitsnews.com', dict(size=15))
plt.text(0.75, -0.7, 'Other sites', dict(size=15))
plt.title("News Source (domain)", fontsize=25)
plt.savefig('newssourcepie.png', bbox_inches='tight', dpi = 300)

# Sentiment Analysis using VADER and Textblob 

In [ ]:
fake=S["type"]=="fake"    # wrong code 

In [ ]:
fake.shape                # wrong code

In [ ]:
fake=S[S["type"]=="fake"]

In [ ]:
fake.shape

In [ ]:
fakecontent = []
for index, row in fake.iterrows():
    fakecontent.append(row['content'])      # content is the column name

In [ ]:
fakecontent

In [ ]:
print (len(fakecontent))

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

In [ ]:
sia=sia()

In [ ]:
results=[]
for i in fakecontent:
    pol_score=sia.polarity_scores(i)
    #print(pol_score)
    pol_score["content"]=i
    #print (i)
    results.append(pol_score)

In [3]:
results

NameError: name 'results' is not defined

In [4]:
fakedf=pd.DataFrame.from_records(results)

NameError: name 'results' is not defined

In [5]:
fakedf.head(n=6)

NameError: name 'fakedf' is not defined

In [6]:
fakedf['label'] = 0
fakedf.loc[fakedf['compound'] > 0.2, 'label'] = 1
fakedf.loc[fakedf['compound'] < -0.2, 'label'] = -1

NameError: name 'fakedf' is not defined

In [7]:
fakedf.label.value_counts()

NameError: name 'fakedf' is not defined

In [8]:
!pip install textblob
from textblob import TextBlob

# Cleaning of the text

In [9]:
S.head()

NameError: name 'S' is not defined

# Removing the punctuations and stemming the words 

In [ ]:
import string

In [ ]:
punctuation=string.punctuation

In [ ]:
def remove_punc(row):
    tokens=row.split()
    
    new_tokens=[w for w in tokens if w.isalpha()]
    new_tokens=[w for w in new_tokens if w not in punctuation]
    return new_tokens

In [ ]:
S["content_w/o_punc"]=S.content.apply(remove_punc)     # content is the column name

In [ ]:
S.head()

In [52]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn

In [53]:
ps=PorterStemmer()

In [54]:
def stemmed_content(ro):
    content=ro["content_w/o_punc"]
    stemmed_list=[ps.stem(words) for words in content]
    return stemmed_list

In [55]:
S["stemmed_words"]=S.apply(stemmed_content,axis=1)

In [56]:
S.head()

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,meta_keywords,meta_description,source,content_w/o_punc,stemmed_words
0,454455,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Google HIPAA Compliance under Question in Cong...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Google HIPAA Compliance under Question in Cong...,Coup Media Group-Scitech,[''],NaN,NaN,"[Google, HIPAA, Compliance, under, Question, i...","[googl, hipaa, complianc, under, question, in,..."
1,6929803,beforeitsnews.com,fake,http://beforeitsnews.com/alternative/2016/01/a...,Ambassador Leo Emil Wanta Round Five with Kare...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Ambassador Leo Emil Wanta Round Five with Kare...,NaN,[''],NaN,NaN,"[Ambassador, Leo, Emil, Wanta, Round, Five, wi...","[ambassador, leo, emil, wanta, round, five, wi..."
2,4460967,beforeitsnews.com,fake,http://beforeitsnews.com/libertarian/2016/06/h...,Today at the OECD Ministerial Meeting on the D...,2017-11-27T01:14:08.7454,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"How Do We Achieve an Open, Secure, Trustworthy...",Electronic Frontier Foundation,[''],NaN,NaN,"[Today, at, the, OECD, Ministerial, Meeting, o...","[today, at, the, oecd, ministeri, meet, on, th..."
3,7002411,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Best Recruitment Marketing Articles of the Wee...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Best Recruitment Marketing Articles of the Wee...,Chris Brablc,[''],NaN,NaN,"[Best, Recruitment, Marketing, Articles, of, t...","[best, recruit, market, articl, of, the, week,..."
4,2920128,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,(ACC VIDEO) Given Dem leadership’s embrace of ...,2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,August 26: National Dog Day – annually!,NaN,[''],NaN,NaN,"[Given, Dem, embrace, of, the, on, and, what, ...","[given, dem, embrac, of, the, on, and, what, w..."


# Remove stopwords

In [57]:
stopwords=set(stopwords.words("english"))

In [58]:
def remove_stopwords(ro):
    tokens=ro["stemmed_words"]
    new_tokens=[w for w in tokens if w not in stopwords]
    return new_tokens

In [59]:
S["noStopStemmedwords"]=S.apply(remove_stopwords,axis=1)

In [60]:
S.head()

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,meta_keywords,meta_description,source,content_w/o_punc,stemmed_words,noStopStemmedwords
0,454455,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Google HIPAA Compliance under Question in Cong...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Google HIPAA Compliance under Question in Cong...,Coup Media Group-Scitech,[''],NaN,NaN,"[Google, HIPAA, Compliance, under, Question, i...","[googl, hipaa, complianc, under, question, in,...","[googl, hipaa, complianc, question, congress, ..."
1,6929803,beforeitsnews.com,fake,http://beforeitsnews.com/alternative/2016/01/a...,Ambassador Leo Emil Wanta Round Five with Kare...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Ambassador Leo Emil Wanta Round Five with Kare...,NaN,[''],NaN,NaN,"[Ambassador, Leo, Emil, Wanta, Round, Five, wi...","[ambassador, leo, emil, wanta, round, five, wi...","[ambassador, leo, emil, wanta, round, five, ka..."
2,4460967,beforeitsnews.com,fake,http://beforeitsnews.com/libertarian/2016/06/h...,Today at the OECD Ministerial Meeting on the D...,2017-11-27T01:14:08.7454,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"How Do We Achieve an Open, Secure, Trustworthy...",Electronic Frontier Foundation,[''],NaN,NaN,"[Today, at, the, OECD, Ministerial, Meeting, o...","[today, at, the, oecd, ministeri, meet, on, th...","[today, oecd, ministeri, meet, digit, economi,..."
3,7002411,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Best Recruitment Marketing Articles of the Wee...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Best Recruitment Marketing Articles of the Wee...,Chris Brablc,[''],NaN,NaN,"[Best, Recruitment, Marketing, Articles, of, t...","[best, recruit, market, articl, of, the, week,...","[best, recruit, market, articl, week, high, fi..."
4,2920128,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,(ACC VIDEO) Given Dem leadership’s embrace of ...,2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,August 26: National Dog Day – annually!,NaN,[''],NaN,NaN,"[Given, Dem, embrace, of, the, on, and, what, ...","[given, dem, embrac, of, the, on, and, what, w...","[given, dem, embrac, know, bill, clinton, Al, ..."


# Rejoining the words

In [61]:
def rejoin(ro):
    tokens=ro["noStopStemmedwords"]
    joined_text=(" ".join(tokens))
    return joined_text

In [62]:
S["joined_words"]=S.apply(rejoin,axis=1)

In [63]:
S.head()

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,meta_keywords,meta_description,source,content_w/o_punc,stemmed_words,noStopStemmedwords,joined_words
0,454455,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Google HIPAA Compliance under Question in Cong...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Google HIPAA Compliance under Question in Cong...,Coup Media Group-Scitech,[''],NaN,NaN,"[Google, HIPAA, Compliance, under, Question, i...","[googl, hipaa, complianc, under, question, in,...","[googl, hipaa, complianc, question, congress, ...",googl hipaa complianc question congress reader...
1,6929803,beforeitsnews.com,fake,http://beforeitsnews.com/alternative/2016/01/a...,Ambassador Leo Emil Wanta Round Five with Kare...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Ambassador Leo Emil Wanta Round Five with Kare...,NaN,[''],NaN,NaN,"[Ambassador, Leo, Emil, Wanta, Round, Five, wi...","[ambassador, leo, emil, wanta, round, five, wi...","[ambassador, leo, emil, wanta, round, five, ka...",ambassador leo emil wanta round five karen jud...
2,4460967,beforeitsnews.com,fake,http://beforeitsnews.com/libertarian/2016/06/h...,Today at the OECD Ministerial Meeting on the D...,2017-11-27T01:14:08.7454,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"How Do We Achieve an Open, Secure, Trustworthy...",Electronic Frontier Foundation,[''],NaN,NaN,"[Today, at, the, OECD, Ministerial, Meeting, o...","[today, at, the, oecd, ministeri, meet, on, th...","[today, oecd, ministeri, meet, digit, economi,...",today oecd ministeri meet digit economi global...
3,7002411,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Best Recruitment Marketing Articles of the Wee...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Best Recruitment Marketing Articles of the Wee...,Chris Brablc,[''],NaN,NaN,"[Best, Recruitment, Marketing, Articles, of, t...","[best, recruit, market, articl, of, the, week,...","[best, recruit, market, articl, week, high, fi...",best recruit market articl week high five take...
4,2920128,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,(ACC VIDEO) Given Dem leadership’s embrace of ...,2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,August 26: National Dog Day – annually!,NaN,[''],NaN,NaN,"[Given, Dem, embrace, of, the, on, and, what, ...","[given, dem, embrac, of, the, on, and, what, w...","[given, dem, embrac, know, bill, clinton, Al, ...",given dem embrac know bill clinton Al nanci pe...


# Creating CountVectorizer

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

In [65]:
S

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,meta_keywords,meta_description,source,content_w/o_punc,stemmed_words,noStopStemmedwords,joined_words
0,454455,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Google HIPAA Compliance under Question in Cong...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Google HIPAA Compliance under Question in Cong...,Coup Media Group-Scitech,[''],NaN,NaN,"[Google, HIPAA, Compliance, under, Question, i...","[googl, hipaa, complianc, under, question, in,...","[googl, hipaa, complianc, question, congress, ...",googl hipaa complianc question congress reader...
1,6929803,beforeitsnews.com,fake,http://beforeitsnews.com/alternative/2016/01/a...,Ambassador Leo Emil Wanta Round Five with Kare...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Ambassador Leo Emil Wanta Round Five with Kare...,NaN,[''],NaN,NaN,"[Ambassador, Leo, Emil, Wanta, Round, Five, wi...","[ambassador, leo, emil, wanta, round, five, wi...","[ambassador, leo, emil, wanta, round, five, ka...",ambassador leo emil wanta round five karen jud...
2,4460967,beforeitsnews.com,fake,http://beforeitsnews.com/libertarian/2016/06/h...,Today at the OECD Ministerial Meeting on the D...,2017-11-27T01:14:08.7454,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"How Do We Achieve an Open, Secure, Trustworthy...",Electronic Frontier Foundation,[''],NaN,NaN,"[Today, at, the, OECD, Ministerial, Meeting, o...","[today, at, the, oecd, ministeri, meet, on, th...","[today, oecd, ministeri, meet, digit, economi,...",today oecd ministeri meet digit economi global...
3,7002411,beforeitsnews.com,fake,http://beforeitsnews.com/science-and-technolog...,Best Recruitment Marketing Articles of the Wee...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Best Recruitment Marketing Articles of the Wee...,Chris Brablc,[''],NaN,NaN,"[Best, Recruitment, Marketing, Articles, of, t...","[best, recruit, market, articl, of, the, week,...","[best, recruit, market, articl, week, high, fi...",best recruit market articl week high five take...
4,2920128,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,(ACC VIDEO) Given Dem leadership’s embrace of ...,2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,August 26: National Dog Day – annually!,NaN,[''],NaN,NaN,"[Given, Dem, embrace, of, the, on, and, what, ...","[given, dem, embrac, of, the, on, and, what, w...","[given, dem, embrac, know, bill, clinton, Al, ...",given dem embrac know bill clinton Al nanci pe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227,8691437,nytimes.com,reliable,https://www.nytimes.com/2003/10/20/nyregion/ta...,"''They study hard, they will have more of an e...",2018-02-11 00:38:10.677356,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,Taking Lessons From Another Culture; For Stude...,Michael Luo,"['NEW YORK CITY', 'EDUCATION AND SCHOOLS', 'CH...",Chinese- and Korean-run cram schools in New Yo...,nytimes,"[study, they, will, have, more, of, an, so, le...","[studi, they, will, have, more, of, an, so, le...","[studi, learn, said, wait, son, get, norma, ha...",studi learn said wait son get norma ha spent l...
3228,9143736,nytimes.com,reliable,https://www.nytimes.com/2008/05/14/us/politics...,With Mr. Obama still solidly ahead of Mrs. Cli...,2018-02-11 00:42:30.403003,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,Clinton Beats Obama Handily in West Virginia,Patrick Healy,"['Presidential Election of 2008', 'West Virgin...",Race was an unusually salient factor in Senato...,nytimes,"[With, Obama, still, solidly, ahead, of, Clint...","[with, obama, still, solidli, ahead, of, clint...","[obama, still, solidli, ahead, clinton, deleg,...",obama still solidli ahead clinton deleg west v...
3229,8215940,www.vice.com,reliable,http://www.vice.com/en_uk/read/sun-ra-poetry-j...,What

In [66]:
A=S["joined_words"]

In [67]:
cv=CountVectorizer(stop_words="english")

In [68]:
Se=cv.fit_transform(A)

In [69]:
print(Se)

  (0, 10604)	3
  (0, 11802)	3
  (0, 5269)	2
  (0, 21139)	2
  (0, 5377)	3
  (0, 21489)	1
  (0, 26484)	1
  (0, 26472)	1
  (0, 25289)	1
  (0, 248)	1
  (0, 2716)	1
  (0, 2836)	2
  (0, 23350)	6
  (0, 8592)	1
  (0, 25246)	1
  (0, 4404)	1
  (0, 20725)	2
  (0, 26332)	1
  (0, 9132)	1
  (0, 229)	1
  (0, 28483)	2
  (0, 11484)	1
  (0, 24011)	1
  (0, 11483)	1
  (0, 21823)	1
  :	:
  (3231, 12643)	1
  (3231, 22510)	2
  (3231, 5213)	1
  (3231, 6458)	1
  (3231, 5973)	1
  (3231, 29455)	1
  (3231, 18945)	1
  (3231, 3306)	1
  (3231, 1677)	5
  (3231, 28552)	1
  (3231, 8023)	1
  (3231, 26238)	2
  (3231, 20668)	1
  (3231, 19144)	1
  (3231, 27127)	1
  (3231, 26882)	1
  (3231, 19815)	1
  (3231, 13145)	1
  (3231, 13143)	1
  (3231, 28569)	2
  (3231, 9625)	2
  (3231, 28563)	2
  (3231, 25915)	2
  (3231, 25913)	2
  (3231, 26884)	1


In [288]:
print(cv.get_feature_names()[:100])

['aa', 'aaa', 'aam', 'aan', 'aanbreken', 'aandacht', 'aangenomen', 'aangeven', 'aanpass', 'aantal', 'aanyc', 'aapl', 'aard', 'aari', 'aaron', 'aavp', 'ab', 'aback', 'abandon', 'abari', 'abas', 'abat', 'abattoir', 'abaya', 'abb', 'abba', 'abbi', 'abbott', 'abbrevi', 'abc', 'abdallah', 'abdel', 'abdi', 'abdic', 'abdict', 'abdifatah', 'abdirahman', 'abdirizak', 'abdomin', 'abdubakir', 'abduboi', 'abduct', 'abducte', 'abdul', 'abdulahi', 'abdulaziz', 'abdullaev', 'abdullah', 'abdullayeva', 'abdumajid', 'abdushakir', 'abe', 'abedin', 'abedini', 'abernathi', 'aberr', 'abet', 'abey', 'abhor', 'abhorr', 'abid', 'abigail', 'abil', 'abingdon', 'abingworth', 'abject', 'abjur', 'abl', 'ablat', 'ablaz', 'ableist', 'ableman', 'abli', 'ablyazov', 'abn', 'abner', 'abnorm', 'abo', 'aboard', 'abod', 'abolish', 'abolit', 'abolitionist', 'abomin', 'aborigin', 'abort', 'abortionist', 'abou', 'aboulian', 'abound', 'abov', 'abrad', 'abraham', 'abrahamson', 'abram', 'abramson', 'abrandt', 'abras', 'abreast', 

In [159]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [160]:
tfidf=TfidfVectorizer()


In [290]:
vc=tfidf.fit_transform(A).toarray()

In [291]:
print(vc)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [292]:
1 in vc

False

In [293]:
2 in vc

False

In [294]:
0.5 in vc

False

In [295]:
tfidf.vocabulary_

{'googl': 10660,
 'hipaa': 11860,
 'complianc': 5295,
 'question': 21236,
 'congress': 5404,
 'reader': 21588,
 'think': 26598,
 'thi': 26584,
 'stori': 25397,
 'add': 249,
 'two': 27439,
 'bitcoin': 2736,
 'blockchain': 2856,
 'search': 23449,
 'exce': 8633,
 'stock': 25354,
 'chang': 4429,
 'privaci': 20821,
 'term': 26443,
 'field': 9174,
 'may': 16226,
 'actual': 230,
 'violat': 28611,
 'healthcar': 11541,
 'sign': 24115,
 'health': 11540,
 'relat': 21922,
 'result': 22144,
 'infring': 12817,
 'boycott': 3188,
 'tri': 27152,
 'uncensor': 27637,
 'anonym': 1039,
 'differ': 6977,
 'media': 16408,
 'sourc': 24772,
 'report': 22050,
 'member': 16511,
 'express': 8734,
 'reserv': 22098,
 'new': 18021,
 'hous': 12153,
 'ha': 11142,
 'caught': 4237,
 'say': 23181,
 'handl': 11296,
 'sensit': 23604,
 'medic': 16415,
 'insur': 12958,
 'portabl': 20487,
 'account': 166,
 'read': 21587,
 'full': 9892,
 'coup': 5768,
 'ambassador': 805,
 'leo': 14907,
 'emil': 8113,
 'wanta': 28896,
 'round': 

In [296]:
len(vc)

3232

In [298]:
from sklearn.feature_extraction import text

In [299]:
print(text.ENGLISH_STOP_WORDS)

frozenset({'who', 'could', 'without', 'whereafter', 'everything', 'above', 'hereafter', 'a', 'and', 'although', 'serious', 'throughout', 'else', 'this', 'thereafter', 'she', 'bill', 'within', 'nowhere', 'other', 'they', 'four', 'everyone', 'ourselves', 'de', 'perhaps', 'me', 'toward', 'couldnt', 'towards', 'fifteen', 'not', 'many', 'except', 'bottom', 'seems', 'mill', 'while', 'my', 'how', 'cannot', 'anyway', 'further', 'therein', 'alone', 'nothing', 'done', 'being', 'whence', 'but', 'any', 'mostly', 'something', 'yourselves', 'must', 'always', 'than', 'every', 'your', 'becoming', 'herself', 'thick', 'fill', 'indeed', 'same', 'someone', 'whoever', 'see', 'among', 'afterwards', 'detail', 'ours', 'sincere', 'off', 'put', 'still', 'these', 'few', 'third', 'un', 'ie', 'whereupon', 'some', 'anyone', 'yourself', 'both', 'up', 'is', 'much', 'top', 'latterly', 'can', 'those', 'below', 'onto', 'at', 'if', 'their', 'however', 'mine', 'con', 'so', 'noone', 'well', 'even', 'therefore', 'eleven', '